In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
!wget http://mirror.coggle.club/uci-smsspamcollection/SMSSpamCollection

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/lyz/.wget-hsts'. HSTS will be disabled.
--2021-01-17 01:12:37--  http://mirror.coggle.club/uci-smsspamcollection/SMSSpamCollection
正在解析主机 mirror.coggle.club (mirror.coggle.club)... 101.72.205.223
正在连接 mirror.coggle.club (mirror.coggle.club)|101.72.205.223|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 477907 (467K) [application/octet-stream]
正在保存至: “SMSSpamCollection”

SMSSpamCollection   100%[===================>] 466.71K  2.04MB/s    用时 0.2s    

2021-01-17 01:12:38 (2.04 MB/s) - 已保存 “SMSSpamCollection” [477907/477907])



In [2]:
data = spark.read.csv("./SMSSpamCollection",inferSchema=True,sep='\t')

In [3]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length',length(data['text']))

In [9]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [11]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

# SPARK SQL

In [12]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [13]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [14]:
from pyspark.ml.classification import NaiveBayes

In [15]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [16]:
from pyspark.ml import Pipeline

In [17]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,
                                  stopremove,count_vec,idf,clean_up])

In [18]:
cleaner = data_prep_pipe.fit(data)

In [19]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [20]:
clean_data = clean_data.select(['label','features'])

In [21]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [22]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [23]:
spam_predictor = nb.fit(training)

In [24]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [25]:
test_results = spam_predictor.transform(testing)

In [26]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,8,1...|[-875.83092790061...|[1.0,2.5355025528...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-662.39105297475...|[1.0,8.6423785920...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-538.18939586954...|[1.0,1.2087996882...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-872.86121197853...|[1.0,6.7947973377...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-445.54813314487...|[1.0,5.1216397083...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-760.79576696251...|[1.0,7.1220946172...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-998.78993591092...|[1.0,3.0779053978...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-616.47832814891...|[0.99207855280157...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1142.0015001984...|[1.66193647649476...|       1.0|
|  0.0|(13424,[0

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9270963184493275
